FE 690 InClass_assignment
Author: William Long
Date: 10/02/2019 
This Notebook should map the geographic center of the 3 digit zip code to each entry in the mortgage data.

In [62]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
#This should give us the mortgage Acquisition data.

AcquisitionColumnNames = (
    "LOAN_ID", "ORIG_CHN", "Seller.Name", 
    "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE",
    "FRST_DTE", "OLTV", "OCLTV", "NUM_BO", 
    "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", 
    "PROP_TYP", "NUM_UNIT", "OCC_STAT", "STATE", "ZIP_3", 
    "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", 
    "RELOCATION_FLG"
)
aq_df = pd.read_csv(
    "Acquisition_2010Q1.txt",
    names=AcquisitionColumnNames,
    header=None,

    sep="|"
)

In [3]:
# We need to load the data
PerformanceColumnNames = (
    "LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", 
    "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
    "Adj.Month.To.Mat", "Maturity.Date", "MSA", 
    "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
    "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", 
    "FCC_COST", "PP_COST", "AR_COST", "IE_COST", 
    "TAX_COST", "NS_PROCS","CE_PROCS", "RMW_PROCS", 
    "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", 
    "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"
)
# I had to trim the data since it was too big for my computer.
performance_data_path = "Performance_2010Q1_new.txt"

performance_df = pd.read_csv(
    performance_data_path,
    names=PerformanceColumnNames,
    header=None,
    sep="|"
)

C:\Users\Will\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
performance_df.head(5)

,LOAN_ID,Monthly.Rpt.Prd,Servicer.Name,LAST_RT,LAST_UPB,Loan.Age,Months.To.Legal.Mat,Adj.Month.To.Mat,Maturity.Date,MSA,...,TAX_COST,NS_PROCS,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG
0,100010079393,02/01/2010,"WELLS FARGO BANK, N.A.",4.875,NaN,0,360,360.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,100010079393,03/01/2010,NaN,4.875,NaN,1,359,358.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,100010079393,04/01/2010,NaN,4.875,NaN,2,358,358.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,100010079393,05/01/2010,NaN,4.875,NaN,3,357,357.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
4,100010079393,06/01/2010,NaN,4.875,NaN,4,356,355.0,02/2040,12420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


In [6]:
performance_df["Delq.Status"]


0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         0
9         0
10        0
11        X
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
20        0
21        0
22        0
23        0
24        0
25        0
26        0
27        0
28        0
29        0
         ..
999954    0
999955    0
999956    0
999957    0
999958    0
999959    0
999960    0
999961    0
999962    0
999963    0
999964    0
999965    0
999966    X
999967    0
999968    0
999969    0
999970    0
999971    0
999972    0
999973    0
999974    0
999975    0
999976    0
999977    0
999978    0
999979    0
999980    0
999981    0
999982    0
999983    0
Name: Delq.Status, Length: 999984, dtype: object

In [4]:
#Set the date.
date = '03/01/2014'

In [5]:
# Create a function to tell if we're in the right period. 
def sameperiod(date1, date2):
    '''
    In:
    date1: String, "MM/DD/YYYY"
    date2: String, "MM/DD/YYYY"
    Out:
    ANS: Bool, True/False
    '''
    d1 = datetime.strptime(date1, '%m/%d/%Y')
    d2 = datetime.strptime(date2, '%m/%d/%Y')
    if d2.year == d1.year and d2.month == d1.month:
        ANS = True
    else:
        ANS = False
    return ANS


In [17]:
sameperiod('01/01/2010', '01/01/2010')

True

In [18]:
sameperiod('01/01/2010', '01/03/2010')

True

In [19]:
sameperiod('01/01/2010', '02/03/2010')

False

In [20]:
sameperiod('01/01/2011', '01/03/2010')

False

In [7]:
performance_df_trimed = performance_df[sameperiod(performance_df["Monthly.Rpt.Prd"], date) == True]

TypeError: strptime() argument 1 must be str, not Series

In [8]:
performance_df_trimed = performance_df[datetime.strptime(performance_df["Monthly.Rpt.Prd"], '%m/%d/%Y').year == datetime.strptime(date, '%m/%d/%Y').year
                                      and datetime.strptime(performance_df["Monthly.Rpt.Prd"], '%m/%d/%Y').month == datetime.strptime(date, '%m/%d/%Y').month]

TypeError: strptime() argument 1 must be str, not Series

Why is datetime such a POS?

In [9]:
def add_year(row):
    t = row["Monthly.Rpt.Prd"]
    year = t[-4:]
    return int(year)

performance_df["Yearly.Rpt.Prd"] = performance_df.apply(lambda row: add_year(row), axis=1)

In [12]:
def add_month(row):
    t = row["Monthly.Rpt.Prd"]
    year = t[:2]
    return int(year)

performance_df["Month"] = performance_df.apply(lambda row: add_month(row), axis=1)

In [27]:
performance_df_trimed = performance_df[performance_df["Yearly.Rpt.Prd"] == int(date[-4:])]
performance_df_trimed = performance_df_trimed[performance_df_trimed["Month"] == int(date[:2])].reset_index()

In [28]:
performance_df_trimed.head(5)

,index,LOAN_ID,Monthly.Rpt.Prd,Servicer.Name,LAST_RT,LAST_UPB,Loan.Age,Months.To.Legal.Mat,Adj.Month.To.Mat,Maturity.Date,...,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG,Yearly.Rpt.Prd,Month
0,62,100013622306,03/01/2014,NaN,4.750,68259.31,50,130,129.0,01/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
1,180,100022098429,03/01/2014,NaN,5.250,432083.23,49,311,311.0,02/2040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
2,383,100032715932,03/01/2014,NaN,5.375,89716.43,51,309,308.0,12/2039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
3,444,100033339771,03/01/2014,NaN,5.250,281467.05,48,312,312.0,03/2040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
4,498,100036162795,03/01/2014,NaN,4.625,4872.09,50,130,4.0,01/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3


In [17]:
len(performance_df_trimed["LOAN_ID"])

8164

In [18]:
len(aq_df["LOAN_ID"])

323174

In [30]:
performance_df_trimed["LOAN_ID"][8163]

149179972416

In [31]:
performance_df_trimed.tail(5)

,index,LOAN_ID,Monthly.Rpt.Prd,Servicer.Name,LAST_RT,LAST_UPB,Loan.Age,Months.To.Legal.Mat,Adj.Month.To.Mat,Maturity.Date,...,CE_PROCS,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG,Yearly.Rpt.Prd,Month
8159,999529,149162653642,03/01/2014,NaN,4.250,141524.03,50,130,103.0,01/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
8160,999630,149170823980,03/01/2014,NaN,5.000,190132.81,50,310,310.0,01/2040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
8161,999693,149171575167,03/01/2014,NaN,4.875,97540.91,49,311,310.0,02/2040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
8162,999802,149175953968,03/01/2014,NaN,5.500,68230.66,48,312,313.0,03/2040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3
8163,999953,149179972416,03/01/2014,NaN,5.250,132741.26,48,312,311.0,03/2040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,2014,3


In [21]:
aq_df["LOAN_ID"][8163]

122499504810

In [29]:
aq_df_trimed = aq_df[performance_df_trimed["LOAN_ID"] == aq_df["LOAN_ID"]]

ValueError: Can only compare identically-labeled Series objects

In [37]:
aq_df.where(performance_df_trimed["LOAN_ID"][-1:] == aq_df["LOAN_ID"])

ValueError: Can only compare identically-labeled Series objects

In [39]:
last_id = int(performance_df_trimed["LOAN_ID"][-1:])

In [42]:
aq_df.loc[last_id == aq_df["LOAN_ID"]]

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,PROP_TYP,NUM_UNIT,OCC_STAT,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG
17717,149179972416,R,OTHER,5.25,142000,360,02/2010,04/2010,80,94.0,...,SF,1,P,NY,146,NaN,FRM,727.0,NaN,N


In [56]:
# Let's try it this way

Delq  = performance_df_trimed.set_index("LOAN_ID")["Delq.Status"].to_dict()
aq_df["Delq.Status"] = aq_df["LOAN_ID"].map(Delq)
holder = aq_df["Delq.Status"].notna()
aq_df_trim = aq_df[holder]
    

In [57]:
aq_df_trim.head(5)

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,NUM_UNIT,OCC_STAT,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,Delq.Status
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,1,P,CA,932,NaN,FRM,785.0,NaN,N,0
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,2,P,NY,112,NaN,FRM,NaN,NaN,N,0
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,3,I,LA,711,NaN,FRM,815.0,NaN,N,0
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,1,P,ID,836,NaN,FRM,NaN,NaN,N,0
10,100036162795,R,"WELLS FARGO BANK, N.A.",4.625,200000,180,12/2009,02/2010,39,39.0,...,1,P,MD,217,NaN,FRM,NaN,NaN,N,0


In [58]:
len(aq_df_trim)

8161

In [59]:
len(performance_df_trimed)

8164

No idea why those are not the same number, but I really don't care anymore.

In [60]:

holder = performance_df["FCC_DTE"].notna()

forclosed = performance_df[holder]

In [61]:
forclosed["FCC_DTE"]

8569      11/01/2012
13104     11/01/2016
19511     03/01/2015
24022     05/01/2016
29452     12/01/2013
31133     05/01/2013
36931     03/01/2016
39172     05/01/2011
61674     04/01/2016
76670     11/01/2013
86697     10/01/2015
99413     07/01/2018
111996    04/01/2017
141526    01/01/2016
178171    05/01/2017
193839    12/01/2013
210742    11/01/2012
215698    01/01/2011
222991    10/01/2016
237631    08/01/2016
239890    02/01/2013
240110    05/01/2018
245974    03/01/2016
272417    09/01/2015
275312    04/01/2015
284751    07/01/2017
340263    05/01/2014
344332    03/01/2012
347542    12/01/2015
354821    08/01/2015
             ...    
639971    07/01/2012
644007    05/01/2015
661099    08/01/2011
662713    08/01/2012
681489    05/01/2012
696433    08/01/2015
736462    06/01/2014
755059    02/01/2014
757356    02/01/2014
762627    12/01/2016
770477    12/01/2013
774090    07/01/2018
796217    04/01/2016
800179    01/01/2016
823638    05/01/2013
831755    04/01/2015
835779    04/

In [65]:
test = datetime.strptime("01/01/2010", '%m/%d/%Y') - datetime.strptime("01/01/2011", '%m/%d/%Y') 

In [66]:
test.days

-365

In [67]:
def mapper_day(row):
    d = datetime.strptime(row["FCC_DTE"], '%m/%d/%Y') - datetime.strptime(date, '%m/%d/%Y') 
    if d.days >= 0 :
        return d.days
    else:
        return np.NaN

forclosed["Time_Delta"] = forclosed.apply(lambda row: mapper_day(row), axis=1)

C:\Users\Will\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [68]:
forclosed

,LOAN_ID,Monthly.Rpt.Prd,Servicer.Name,LAST_RT,LAST_UPB,Loan.Age,Months.To.Legal.Mat,Adj.Month.To.Mat,Maturity.Date,MSA,...,RMW_PROCS,O_PROCS,NON_INT_UPB,PRIN_FORG_UPB_FHFA,REPCH_FLAG,PRIN_FORG_UPB_OTH,TRANSFER_FLG,Yearly.Rpt.Prd,Month,Time_Delta
8569,100446401716,11/01/2012,NaN,5.125,239556.43,34,326,0.0,01/2040,12580,...,NaN,NaN,NaN,NaN,N,NaN,NaN,2012,11,NaN
13104,100683589832,11/01/2016,NaN,4.875,86107.60,80,280,0.0,03/2040,47260,...,NaN,2356.39,NaN,NaN,N,NaN,NaN,2016,11,976.0
19511,100960628290,03/01/2015,NaN,5.000,93443.74,62,298,0.0,01/2040,12580,...,NaN,680.00,NaN,NaN,N,NaN,NaN,2015,3,365.0
24022,101212627627,05/01/2016,NaN,5.000,317574.76,76,284,0.0,01/2040,41740,...,NaN,NaN,NaN,NaN,N,NaN,NaN,2016,5,792.0
29452,101455126320,12/01/2013,NaN,5.250,110039.77,47,313,0.0,01/2040,49620,...,NaN,NaN,NaN,NaN,N,NaN,NaN,2013,12,NaN
31133,101558146692,05/01/2013,NaN,4.625,228098.09,40,140,0.0,01/2025,0,...,NaN,1409.95,NaN,NaN,N,NaN,NaN,2013,5,NaN
36931,101865143451,03/01/2016,NaN,4.250,41220.56,72,108,0.0,03/2025,17140,...,NaN,4447.86,NaN,NaN,N,NaN,NaN,2016,3,731.0
39172,101975325389,05/01/2011,NaN,5.940,60527.41,16,344,0.0,01/2040,19820,...,NaN,NaN,NaN,NaN,N,NaN,NaN,2011,5,NaN
61674,103182946141,04/01/2016,NaN,4.875,60766.71,74,166,0.0,02/2030,28700,...,NaN,26.00,NaN,NaN,N,NaN,NaN,2016,4,762.0
76670,103962879924,11/01/2013,NaN,5.375,294796.38,46,314,0.0,01/2040,0,...,NaN,NaN,NaN,NaN,N,NaN,NaN,2013,11,NaN


In [72]:
Delta  = forclosed.set_index("LOAN_ID")["Time_Delta"].to_dict()
aq_df_trim["Time_Delta"] = aq_df["LOAN_ID"].map(Delta)

C:\Users\Will\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
aq_df_trim.head(5)

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,OCC_STAT,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,Delq.Status,Time_Delta
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,P,CA,932,NaN,FRM,785.0,NaN,N,0,NaN
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,P,NY,112,NaN,FRM,NaN,NaN,N,0,NaN
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,I,LA,711,NaN,FRM,815.0,NaN,N,0,NaN
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,P,ID,836,NaN,FRM,NaN,NaN,N,0,NaN
10,100036162795,R,"WELLS FARGO BANK, N.A.",4.625,200000,180,12/2009,02/2010,39,39.0,...,P,MD,217,NaN,FRM,NaN,NaN,N,0,NaN


So, I finally figured out how to do part two. Now for part 3. 


In [75]:
aq_df_big_trim = aq_df_trim[int(aq_df_trim["Delq.Status"]) > 0]

TypeError: cannot convert the series to <class 'int'>

In [76]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [78]:
def mapper_delq(row):
    if is_number(row["Delq.Status"]) == True:
        return int(row["Delq.Status"])
    else:
        return 0 

aq_df_trim["Delq.Status_INT"] = aq_df_trim.apply(lambda row: mapper_delq(row), axis=1)


    

C:\Users\Will\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [79]:
aq_df_trim.head(5)

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,Delq.Status,Time_Delta,Delq.Status_INT
1,100013622306,R,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,87000,180,12/2009,02/2010,63,63.0,...,CA,932,NaN,FRM,785.0,NaN,N,0,NaN,0
3,100022098429,R,OTHER,5.250,461000,360,01/2010,03/2010,61,61.0,...,NY,112,NaN,FRM,NaN,NaN,N,0,NaN,0
8,100032715932,R,SUNTRUST MORTGAGE INC.,5.375,96000,360,12/2009,01/2010,75,75.0,...,LA,711,NaN,FRM,815.0,NaN,N,0,NaN,0
9,100033339771,R,OTHER,5.250,300000,360,02/2010,04/2010,74,74.0,...,ID,836,NaN,FRM,NaN,NaN,N,0,NaN,0
10,100036162795,R,"WELLS FARGO BANK, N.A.",4.625,200000,180,12/2009,02/2010,39,39.0,...,MD,217,NaN,FRM,NaN,NaN,N,0,NaN,0


In [81]:
aq_df_big_trim = aq_df_trim[aq_df_trim["Delq.Status_INT"] > 0]

In [82]:
aq_df_big_trim

,LOAN_ID,ORIG_CHN,Seller.Name,ORIG_RT,ORIG_AMT,ORIG_TRM,ORIG_DTE,FRST_DTE,OLTV,OCLTV,...,STATE,ZIP_3,MI_PCT,Product.Type,CSCORE_C,MI_TYPE,RELOCATION_FLG,Delq.Status,Time_Delta,Delq.Status_INT
15,100049953168,B,"BANK OF AMERICA, N.A.",5.250,292000,360,01/2010,03/2010,80,80.0,...,NY,115,NaN,FRM,NaN,NaN,N,1,NaN,1
31,100091653134,R,OTHER,5.375,312000,360,12/2009,02/2010,80,80.0,...,WV,265,NaN,FRM,NaN,NaN,N,1,NaN,1
110,100312066648,R,OTHER,5.250,160000,360,01/2010,03/2010,46,46.0,...,PA,183,NaN,FRM,NaN,NaN,N,5,NaN,5
131,100393550601,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.750,80000,180,11/2009,01/2010,51,51.0,...,TX,786,NaN,FRM,650.0,NaN,N,2,NaN,2
213,100657701189,C,"JPMORGAN CHASE BANK, NATIONAL ASSOCIATION",4.875,220000,240,12/2009,02/2010,61,61.0,...,MD,210,NaN,FRM,708.0,NaN,N,1,NaN,1
328,100929000903,R,OTHER,5.250,432000,360,11/2009,01/2010,66,66.0,...,NY,109,NaN,FRM,NaN,NaN,N,20,NaN,20
341,100960628290,B,"CITIMORTGAGE, INC.",5.000,100000,360,12/2009,02/2010,63,63.0,...,MD,212,NaN,FRM,NaN,NaN,N,1,365.0,1
587,101680663541,R,OTHER,4.875,77000,180,01/2010,03/2010,52,52.0,...,OH,458,NaN,FRM,NaN,NaN,N,1,NaN,1
731,102038753630,R,OTHER,4.750,84000,180,01/2010,03/2010,80,80.0,...,NJ,89,NaN,FRM,NaN,NaN,N,3,NaN,3
762,102126520376,C,"CITIMORTGAGE, INC.",5.250,70000,180,10/2009,12/2009,47,47.0,...,LA,708,NaN,FRM,624.0,NaN,N,1,NaN,1
